In [35]:
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
from crewai.tools import tool
import yfinance as yf


class Tools:

    # no need self but use @tools
    @tool("One month stock price history")
    #! """<<explanation>>""", return is very important
    def stock_price(ticker):
        """Useful to get a month's worth of stock price data as CSV.
        The input of this tool should be a ticker, for example AAPL, NET, TSLA, etc...

        Args:
            ticker (str): Ticker of the company stock

        Returns:
            str: Stock price history for a month in CSV format
        """
        stock = yf.Ticker(ticker)
        return stock.history(period="1mo").to_csv()

    @tool("Stock news URLs")
    def stock_news(ticker):
        """
        Useful to get URLs of recent news articles related to a stock.
        The input of this tool should be a ticker, for example AAPL, NET, TSLA, etc...

        Args:
            ticker (str): Ticker of the company stock

        Returns:
            str: A list of news article URLs, separated by newlines
        """
        stock = yf.Ticker(ticker)
        return list(map(lambda x: x["link"], stock.news))

    @tool("Company's income statement")
    def income_stmt(ticker):
        """
        Useful to get the company's income statement as CSV.
        The input of this tool should be a ticker, for example AAPL, NET, TSLA, etc...

        Args:
            ticker (str): Ticker of the company stock

        Returns:
            str: Income statement data in CSV format
        """
        stock = yf.Ticker(ticker)
        return stock.income_stmt.to_csv()

    @tool("Balance sheet")
    def balance_sheet(ticker):
        """
        Useful to get the company's balance sheet as CSV.
        The input of this tool should be a ticker, for example AAPL, NET, TSLA, etc...

        Args:
            ticker (str): Ticker of the company stock

        Returns:
            str: Balance sheet data in CSV format
        """
        stock = yf.Ticker(ticker)
        return stock.balance_sheet.to_csv()

    @tool("Get insider transactions")
    def insider_transactions(ticker):
        """
        Useful to get insider transaction data for a company as CSV.
        The input of this tool should be a ticker, for example AAPL, NET, TSLA, etc...

        Args:
            ticker (str): Ticker of the company stock

        Returns:
            str: Insider transaction data in CSV format
        """
        stock = yf.Ticker(ticker)
        return stock.insider_transactions.to_csv()

In [37]:
#
# * check tool output
# stock = yf.Ticker("AAPL")
# print(stock.history(period="1mo").to_csv())
# print(stock.history(period="1mo"))

In [38]:
from crewai import Agent
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


class Agents:
    def __init__(self):
        self.search_tool = SerperDevTool()
        self.scrape_tool = ScrapeWebsiteTool()

        self.researcher = Agent(
            role="Researcher",
            goal="Gathers, interprets and summarizes vasts amounts of data to provide a comprehensive overview of the sentiment and news surrounding a stock.",
            backstory="You're skilled in gathering and interpreting data from various sources to give a complete picture of a stock's sentiment and news. You read each data source carefuly and extract the most important information. Your insights are crucial for making informed investment decisions.",
            verbose=True,
            tools=[
                Tools.stock_news,
                self.search_tool,
                self.scrape_tool,
            ],
        )
        self.technical_analyst = Agent(
            role="Technical Analyst",
            goal="Analyses the movements of a stock and provides insights on trends, entry points, resistance and support levels.",
            backstory="An expert in technical analysis, you're known for your ability to predict stock movements and trends based on historical data. You provide valuable insights to your customers.",
            verbose=True,
            tools=[Tools.stock_price],
        )

        self.financial_analyst = Agent(
            role="Financial Analyst",
            goal="Uses financial statements, insider trading data, and other financial metrics to evaluate a stock's financial health and performance.",
            backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients. You look at a company's financial health, market sentiment, and qualitative data to make informed recommendations.",
            verbose=True,
            tools=[
                Tools.balance_sheet,
                Tools.income_stmt,
                Tools.insider_transactions,
            ],
        )

        self.hedge_fund_manager = Agent(
            role="Hedge Fund Manager",
            goal="Manages a portfolio of stocks and makes strategic investment decisions to maximize returns using insights from financial analysts, technical analysts, and researchers.",
            backstory="You're a seasoned hedge fund manager with a proven track record of making profitable investment decisions. You're known for your ability to manage risk and maximize returns for your clients.",
            verbose=True,
        )

In [39]:
from crewai import Task


class Tasks:

    def research(self, agent):
        return Task(
            description="Gather and analyze the latest news and market sentiment surrounding the stock of {company}. Provide a summary of the news and any notable shifts in market sentiment.",
            expected_output=f"Your final answer MUST be a detailed summary of the news and market sentiment surrounding the stock. Include any notable shifts in market sentiment and provide insights on how these factors could impact the stock's performance.",
            agent=agent,
            output_file="stock_news.md",
        )

    def technical_analysis(self, agent):
        return Task(
            description="Conduct a detailed technical analysis of the price movements of {company}'s stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.",
            expected_output=f"Your final answer MUST be a detailed technical analysis report that includes key support and resistance levels, chart patterns, and technical indicators. Provide insights on potential entry points, price targets, and any other relevant information that could help your customer make informed investment decisions.",
            agent=agent,
            output_file="technical_analysis.md",
        )

    def financial_analysis(self, agent):
        return Task(
            description="Analyze {company}'s financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            expected_output=f"Your final answer MUST be a detailed financial analysis report that includes insights on the company's financial health, performance, and valuation. Provide an overview of the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.",
            agent=agent,
            output_file="financial_analysis.md",
        )

    def investment_recommendation(self, agent, context):
        return Task(
            description="Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for {company}'s stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.",
            expected_output=f"Your final answer MUST be a detailed investment recommendation report to BUY or SELL the stock that includes your analysis of the stock's potential risks and rewards. Provide a clear rationale for your recommendation based on the research, technical analysis, and financial analysis reports.",
            agent=agent,
            context=context,
            output_file="investment_recommendation.md",
        )

In [ ]:
from crewai import Crew, LLM
from crewai.process import Process
from langchain_openai import ChatOpenAI

agents = Agents()
tasks = Tasks()

# initialize agents
researcher = agents.researcher
technical_analyst = agents.technical_analyst
financial_analyst = agents.financial_analyst
hedge_fund_managet = agents.hedge_fund_manager

# initialize tasks
research_task = tasks.research(researcher)
technical_task = tasks.technical_analysis(technical_analyst)
financial_task = tasks.financial_analysis(financial_analyst)
recommend_task = tasks.investment_recommendation(
    hedge_fund_managet,
    [
        research_task,
        technical_task,
        financial_task,
    ],
)

crew = Crew(
    agents=[
        researcher,
        technical_analyst,
        financial_analyst,
        hedge_fund_managet,
    ],
    tasks=[
        research_task,
        technical_task,
        financial_task,
        recommend_task,
    ],
    #! hierarchical doesn't give suitable answers for this crew
    # process=Process.hierarchical,
    # process=Process.sequential,
    # * can change this
    # manager_llm=ChatOpenAI(model="gpt-4.1-nano"),
    # * using local llm by using ollama
    manager_llm=LLM(
        model="ollama/gemma3",
        base_url="http://localhost:11434",
    ),
    # memory uses embeddings -> more cost
    memory=True,
    planning=True,
    # verbose=True,
)
# * check ollama local llm is running -> ok
# print(crew.manager_llm.model)
result = crew.kickoff(
    inputs=dict(
        company="Salesforce",
    ),
)

ollama/gemma3
# Agent: Researcher
## Task: Gather and analyze the latest news and market sentiment surrounding the stock of Salesforce. Provide a summary of the news and any notable shifts in market sentiment.1. Utilize the 'Stock news URLs' tool to gather recent articles related to Salesforce stock by inputting the ticker 'CRM'. This tool will return a list of news article URLs. 
2. Once the URLs are obtained, use the 'Read website content' tool for each URL to scrape and read the content of the articles. 
3. Analyze the gathered information focusing on key points such as announcements, earnings reports, analyst ratings, and market reactions. 
4. Summarize the findings, highlighting notable shifts in market sentiment and key developments that could influence Salesforce's stock performance.
 

I encountered an error while trying to use the tool. This was the error: Tools.stock_news() missing 1 required positional argument: 'ticker'.
 Tool Stock news URLs accepts these inputs: Tool Name

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Technical Analyst
## Task: Conduct a detailed technical analysis of the price movements of Salesforce's stock and trends identify key support and resistance levels, chart patterns, and other technical indicators that could influence the stock's future performance. Use historical price data and technical analysis tools to provide insights on potential entry points and price targets.1. Use the 'One month stock price history' tool to retrieve the last month's worth of Salesforce stock price history by inputting the ticker 'CRM'. 
2. Analyze the historical price data to identify key support and resistance levels. Look for reversal patterns, trendlines, and moving averages. 
3. Identify any prevalent chart patterns such as head and shoulders, cups and handles, or flags that indicate potential price movements. 
4. Utilize technical indicators such as RSI (Relative Strength Index) and MACD (Moving Average Convergence Divergence) to assess trends and signals for potential entry points

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Financial Analyst
## Task: Analyze Salesforce's financial statements, insider trading data, and other financial metrics to evaluate the stock's financial health and performance. Provide insights on the company's revenue, earnings, cash flow, and other key financial metrics. Use financial analysis tools and models to assess the stock's valuation and growth potential.1. Use the 'Balance sheet', 'Company's income statement', and 'Get insider transactions' tools to fetch relevant financial data by inputting the ticker 'CRM'. 
2. Analyze the balance sheet to assess the company’s assets, liabilities, and equity structure. Look for financial health indicators like current ratio, quick ratio, and debt-to-equity ratio. 
3. Review the income statement for insights on revenues, costs, profit margins, and net income trends. Pay attention to year-over-year growth rates and operating income. 
4. Examine insider trading data to understand if insiders are buying or selling shares, which can i

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


# Agent: Hedge Fund Manager
## Task: Based on the research, technical analysis, and financial analysis reports, provide a detailed investment recommendation for Salesforce's stock. Include your analysis of the stock's potential risks and rewards, and provide a clear rationale for your recommendation.1. Compile the analysis reports from the researcher, technical analyst, and financial analyst. 
2. Compare the insights from each analysis, focusing on market sentiment, technical indicators, and financial health metrics. 
3. Analyze the potential risks such as market volatility, competitive pressures, and regulatory challenges. Then evaluate the potential rewards like growth opportunities and market share expansion. 
4. Formulate a clear investment recommendation to either BUY or SELL Salesforce stock based on the comprehensive analysis. 
5. Provide a rationale for the recommendation, clearly linking it to the insights derived from research and analysis to assist decision-making.


# Agent

/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/Users/suajeong/code/gpt-assistants-fullstack/env/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
